This notebook contains code for rendering .png images according to a predefined fsleyes orthographic view, of 3D-FLAIR based MNI normalized TOPUP and EPIC field maps overlayed on a MNI brain regions template. It also uses ffmpeg to make videos of the images.

1. Specify epi_corrections output directory

In [1]:
output_directory_suffix = "2019_07_02"
# On local file system:
corrections_base_directory = "../../epi_corrections_out_" + output_directory_suffix
# On samba share (remote file sytem):
#corrections_base_directory = "/run/user/1000/gvfs/smb-share:server=192.168.1.207,share=hdd3tb1/data/IVS_EPI_BASELINE/epi_corrections_out_" + output_directory_suffix
corrections_base_directory

'../../epi_corrections_out_2019_07_02'

2. Since nICE introduced files with spaces in their file names, 
recursively replace spaces in folders and file names.
The fsleyes rendering bash command did not accept 
spaces in file names and directories. 

In [4]:
from utils import replace_spaces_with_underscore
replace_spaces_with_underscore(corrections_base_directory + "/EPI_raw_DSC")
replace_spaces_with_underscore(corrections_base_directory + "/EPI_applytopup")
replace_spaces_with_underscore(corrections_base_directory + "/EPI_applyepic")

3. Find the MNI-normalized, NordicICE intensity Normalized Relative CBV files. Can be done remotely.

In [2]:
from search import find_field_files

[topup_field_files_e1, \
 epic_field_files_e1, \
 topup_field_files_e2, \
 epic_field_files_e2] = find_field_files(corrections_base_directory)

In [3]:
# A check
print("Equal number of detected field files for topup, and epic correction methods: %r" % \
      (len(topup_field_files_e1) == \
       len(topup_field_files_e2) == \
       len(epic_field_files_e1) == \
       len(epic_field_files_e2)))
print("Number of subject field files: %i" % len(topup_field_files_e1))

Equal number of detected field files for topup, and epic correction methods: True
Number of subject field files: 45


4. Find the corresponding MNI label files 
(should perhaps be identical to each other, but just to make sure..)

In [4]:
from search import find_label_files

[raw_label_files_e1, \
 topup_label_files_e1, \
 epic_label_files_e1, \
 raw_label_files_e2, \
 topup_label_files_e2, \
 epic_label_files_e2] = find_label_files(corrections_base_directory)

In [5]:
# A check
print("Equal number of detected label files for raw (uncorrected), topup, and epic correction methods: %r" % \
      (len(raw_label_files_e1) == \
       len(raw_label_files_e2) == \
       len(topup_label_files_e1) == \
       len(topup_label_files_e2) == \
       len(epic_label_files_e1) == \
       len(epic_label_files_e2)))
print("Number of subject label files: %i" % len(raw_label_files_e1))

Equal number of detected label files for raw (uncorrected), topup, and epic correction methods: True
Number of subject label files: 45


5. For the raw, topup and epic Gradient Echo (GE) and Spin Echo (SE) nrCBV data + MNI regions file, render .png images in a predefined orthographic view, then make a video. __Note:__ The following cells must run within a fsl X environment

In [14]:
%%bash -s "{" ".join(topup_field_files_e1)}" "{" ".join(epic_field_files_e1)}" "{" ".join(topup_field_files_e2)}" "{" ".join(epic_field_files_e2)}" "{" ".join(topup_label_files_e1)}" "{" ".join(epic_label_files_e1)}" "{" ".join(topup_label_files_e2)}" "{" ".join(epic_label_files_e2)}"
IFS=' ' read -r -a topup_field_files_e1 <<< "$1"
IFS=' ' read -r -a epic_field_files_e1 <<< "$2"
IFS=' ' read -r -a topup_field_files_e2 <<< "$3"
IFS=' ' read -r -a epic_field_files_e2 <<< "$4"
IFS=' ' read -r -a topup_label_files_e1 <<< "$5"
IFS=' ' read -r -a epic_label_files_e1 <<< "$6"
IFS=' ' read -r -a topup_label_files_e2 <<< "$7"
IFS=' ' read -r -a epic_label_files_e2 <<< "$8"
render() {
    local -n _field_files=$1
    local -n _label_files=$2
    local -n _render_dir=$3
    
    # Remove existing render directory.
    if [ -d "$_render_dir" ]; then rm -rd "$_render_dir"; fi
    
    mkdir -p "$_render_dir"
    
    for index in "${!_field_files[@]}"
    do
        field_file="${_field_files[index]}"
        label_file="${_label_files[index]}"
        
        field_file_name="$(basename $field_file)"
        label_file_name="$(basename $label_file)"
        
        render_command='fsleyes 
            render 
            --outfile "'$_render_dir'/'output_$(printf "%03d\n" $index)'" 
            --scene ortho 
            --worldLoc 9.918212890625e-05 -18.000099182128906 7.999900817871094 
            --displaySpace 
            '$field_file' 
            --xcentre  0.00000  0.00000 
            --ycentre  0.00000  0.00000 
            --zcentre  0.00000 -0.00000 
            --xzoom 100.0 
            --yzoom 100.0 
            --zzoom 100.0 
            --hideLabels 
            --labelSize 14 
            --layout horizontal 
            --hideCursor 
            --bgColour 0.0 0.0 0.0 
            --fgColour 1.0 1.0 1.0 
            --cursorColour 0.0 1.0 0.0 
            --showColourBar 
            --colourBarLocation top 
            --colourBarLabelSide top-left 
            --performance 3 
            '$label_file' 
            --name '$label_file_name' 
            --overlayType volume 
            --alpha 100.0 
            --brightness 50.0 
            --contrast 50.0 
            --cmap greyscale 
            --negativeCmap greyscale 
            --displayRange 0.0 207.0 
            --clippingRange 0.0 209.07 
            --gamma 0.0 
            --cmapResolution 256 
            --interpolation none 
            --numSteps 100 
            --blendFactor 0.1 
            --smoothing 0 
            --resolution 100 
            --numInnerSteps 10 
            --clipMode intersection 
            --volume 0 
            '$field_file' 
            --name '$field_file_name' 
            --overlayType volume 
            --alpha 79.33333334031825 
            --brightness 54.09163686077013 
            --contrast 49.09314381711701 
            --cmap render3 
            --negativeCmap greyscale 
            --displayRange -6.0 6.0 
            --clippingRange -6.0 12.869085369110108 
            --gamma 0.0 --cmapResolution 256 
            --interpolation none 
            --numSteps 100 
            --blendFactor 0.1 
            --smoothing 0 
            --resolution 100 
            --numInnerSteps 10 
            --clipMode intersection 
            --volume 0'
        echo $render_command >> "$_render_dir/commands.txt"
        #echo $render_command
        eval $render_command
        echo "--Finished rendering volume $index to .png--"
    done
    ffmpeg -framerate 1 -i "$_render_dir"/output_%03d.png -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -c:v libx264 -preset slow -profile:v high -level:v 4.0 -pix_fmt yuv420p -crf 1 "$_render_dir"/video.mp4
    echo "--Finished rendering pngs to video--"
}
render_dir_topup_field_e1="../../epi_corrections_out_2019_07_02/render_fields/TOPUP/e1"
render topup_field_files_e1 topup_label_files_e1 render_dir_topup_field_e1
sleep 2 # Sleep 1 second; this was necessary otherwise the next call of render() function did seem to get emtpy render dir argument...
render_dir_topup_field_e2="../../epi_corrections_out_2019_07_02/render_fields/TOPUP/e2"
render topup_field_files_e2 topup_label_files_e2 render_dir_topup_field_e2

--Finished rendering volume 0 to .png--
--Finished rendering volume 1 to .png--
--Finished rendering volume 2 to .png--
--Finished rendering volume 3 to .png--
--Finished rendering volume 4 to .png--
--Finished rendering volume 5 to .png--
--Finished rendering volume 6 to .png--
--Finished rendering volume 7 to .png--
--Finished rendering volume 8 to .png--
--Finished rendering volume 9 to .png--
--Finished rendering volume 10 to .png--
--Finished rendering volume 11 to .png--
--Finished rendering volume 12 to .png--
--Finished rendering volume 13 to .png--
--Finished rendering volume 14 to .png--
--Finished rendering volume 15 to .png--
--Finished rendering volume 16 to .png--
--Finished rendering volume 17 to .png--
--Finished rendering volume 18 to .png--
--Finished rendering volume 19 to .png--
--Finished rendering volume 20 to .png--
--Finished rendering volume 21 to .png--
--Finished rendering volume 22 to .png--
--Finished rendering volume 23 to .png--
--Finished rendering volum


(wr_coregest_145923_GE-SE_EPI_SSH_v1_32CH_V2_scan_prescan_1001_901_e2_0000_prep_topup_field_postp.nii:31439): Gtk-CRITICAL **: 20:17:06.933: gtk_window_resize: assertion 'width > 0' failed
20:17:07: Debug: Adding duplicate image handler for 'Windows bitmap file'
 WARNING       numpymodule.py   30: <module>        - Unable to load numpy_formathandler accelerator from OpenGL_accelerate

(wr_coregest_120520_GE-SE_EPI_SSH_v1_32CH_V2_scan_prescan_1101_901_e2_0000_prep_topup_field_postp.nii:31461): Gtk-CRITICAL **: 20:17:12.401: gtk_window_resize: assertion 'width > 0' failed
20:17:12: Debug: Adding duplicate image handler for 'Windows bitmap file'
 WARNING       numpymodule.py   30: <module>        - Unable to load numpy_formathandler accelerator from OpenGL_accelerate

(wr_coregest_104123_GE-SE_EPI_SSH_v1_32CH_V2_scan_prescan_1001_901_e2_0000_prep_topup_field_postp.nii:31482): Gtk-CRITICAL **: 20:17:18.081: gtk_window_resize: assertion 'width > 0' failed
20:17:18: Debug: Adding duplicate 

In [15]:
%%bash -s "{" ".join(topup_field_files_e1)}" "{" ".join(epic_field_files_e1)}" "{" ".join(topup_field_files_e2)}" "{" ".join(epic_field_files_e2)}" "{" ".join(topup_label_files_e1)}" "{" ".join(epic_label_files_e1)}" "{" ".join(topup_label_files_e2)}" "{" ".join(epic_label_files_e2)}"
IFS=' ' read -r -a topup_field_files_e1 <<< "$1"
IFS=' ' read -r -a epic_field_files_e1 <<< "$2"
IFS=' ' read -r -a topup_field_files_e2 <<< "$3"
IFS=' ' read -r -a epic_field_files_e2 <<< "$4"
IFS=' ' read -r -a topup_label_files_e1 <<< "$5"
IFS=' ' read -r -a epic_label_files_e1 <<< "$6"
IFS=' ' read -r -a topup_label_files_e2 <<< "$7"
IFS=' ' read -r -a epic_label_files_e2 <<< "$8"
render() {
    local -n _field_files=$1
    local -n _label_files=$2
    local -n _render_dir=$3
    
    # Remove existing render directory.
    if [ -d "$_render_dir" ]; then rm -rd "$_render_dir"; fi
    
    mkdir -p "$_render_dir"
    
    for index in "${!_field_files[@]}"
    do
        field_file="${_field_files[index]}"
        label_file="${_label_files[index]}"
        
        field_file_name="$(basename $field_file)"
        label_file_name="$(basename $label_file)"
        
        render_command='fsleyes 
            render 
            --outfile "'$_render_dir'/'output_$(printf "%03d\n" $index)'" 
            --scene ortho 
            --worldLoc 9.918212890625e-05 -18.000099182128906 7.999900817871094 
            --displaySpace 
            '$field_file' 
            --xcentre  0.00000  0.00000 
            --ycentre  0.00000  0.00000 
            --zcentre  0.00000 -0.00000 
            --xzoom 100.0 
            --yzoom 100.0 
            --zzoom 100.0 
            --hideLabels 
            --labelSize 14 
            --layout horizontal 
            --hideCursor 
            --bgColour 0.0 0.0 0.0 
            --fgColour 1.0 1.0 1.0 
            --cursorColour 0.0 1.0 0.0 
            --showColourBar 
            --colourBarLocation top 
            --colourBarLabelSide top-left 
            --performance 3 
            '$label_file' 
            --name '$label_file_name' 
            --overlayType volume 
            --alpha 100.0 
            --brightness 50.0 
            --contrast 50.0 
            --cmap greyscale 
            --negativeCmap greyscale 
            --displayRange 0.0 207.0 
            --clippingRange 0.0 209.07 
            --gamma 0.0 
            --cmapResolution 256 
            --interpolation none 
            --numSteps 100 
            --blendFactor 0.1 
            --smoothing 0 
            --resolution 100 
            --numInnerSteps 10 
            --clipMode intersection 
            --volume 0 
            '$field_file' 
            --name '$field_file_name' 
            --overlayType volume 
            --alpha 79.33333334031825 
            --brightness 54.09163686077013 
            --contrast 49.09314381711701 
            --cmap render3 
            --negativeCmap greyscale 
            --displayRange -6.0 6.0 
            --clippingRange -6.0 6.505926780700683 
            --gamma 0.0 --cmapResolution 256 
            --interpolation none 
            --numSteps 100 
            --blendFactor 0.1 
            --smoothing 0 
            --resolution 100 
            --numInnerSteps 10 
            --clipMode intersection 
            --volume 0'
        echo $render_command >> "$_render_dir/commands.txt"
        #echo $render_command
        eval $render_command
        echo "--Finished rendering volume $index to .png--"
    done
    ffmpeg -framerate 1 -i "$_render_dir"/output_%03d.png -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -c:v libx264 -preset slow -profile:v high -level:v 4.0 -pix_fmt yuv420p -crf 1 "$_render_dir"/video.mp4
    echo "--Finished rendering pngs to video--"
}
sleep 1
render_dir_epic_field_e1="../../epi_corrections_out_2019_07_02/render_fields/EPIC/e1"
render epic_field_files_e1 epic_label_files_e1 render_dir_epic_field_e1
sleep 2 # Sleep 1 second; this was necessary otherwise the next call of render() function did seem to get emtpy render dir argument...
render_dir_topup_field_e2="../../epi_corrections_out_2019_07_02/render_fields/EPIC/e2"
render epic_field_files_e2 epic_label_files_e2 render_dir_topup_field_e2

--Finished rendering volume 0 to .png--
--Finished rendering volume 1 to .png--
--Finished rendering volume 2 to .png--
--Finished rendering volume 3 to .png--
--Finished rendering volume 4 to .png--
--Finished rendering volume 5 to .png--
--Finished rendering volume 6 to .png--
--Finished rendering volume 7 to .png--
--Finished rendering volume 8 to .png--
--Finished rendering volume 9 to .png--
--Finished rendering volume 10 to .png--
--Finished rendering volume 11 to .png--
--Finished rendering volume 12 to .png--
--Finished rendering volume 13 to .png--
--Finished rendering volume 14 to .png--
--Finished rendering volume 15 to .png--
--Finished rendering volume 16 to .png--
--Finished rendering volume 17 to .png--
--Finished rendering volume 18 to .png--
--Finished rendering volume 19 to .png--
--Finished rendering volume 20 to .png--
--Finished rendering volume 21 to .png--
--Finished rendering volume 22 to .png--
--Finished rendering volume 23 to .png--
--Finished rendering volum


(wr_coregest_displacement_field_e1.nii:32484): Gtk-CRITICAL **: 20:22:25.547: gtk_window_resize: assertion 'width > 0' failed
20:22:25: Debug: Adding duplicate image handler for 'Windows bitmap file'
 WARNING       numpymodule.py   30: <module>        - Unable to load numpy_formathandler accelerator from OpenGL_accelerate

(wr_coregest_displacement_field_e1.nii:32506): Gtk-CRITICAL **: 20:22:31.373: gtk_window_resize: assertion 'width > 0' failed
20:22:31: Debug: Adding duplicate image handler for 'Windows bitmap file'
 WARNING       numpymodule.py   30: <module>        - Unable to load numpy_formathandler accelerator from OpenGL_accelerate

(wr_coregest_displacement_field_e1.nii:32530): Gtk-CRITICAL **: 20:22:36.906: gtk_window_resize: assertion 'width > 0' failed
20:22:37: Debug: Adding duplicate image handler for 'Windows bitmap file'
 WARNING       numpymodule.py   30: <module>        - Unable to load numpy_formathandler accelerator from OpenGL_accelerate

(wr_coregest_displacemen